<a href="https://colab.research.google.com/github/Likhithluck/Capstone-Project/blob/tarun/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# !pip install pdf2image
# !pip install datasets
# !pip install pytesseract
# !pip install pdfplumber
# !apt-get install tesseract-ocr
# !apt-get install libtesseract-dev
# !apt-get install poppler-utils # Install poppler-utils which includes pdfinfo
# pip install camelot-py[cv]


In [52]:
import pandas as panda
from huggingface_hub import login
from datasets import load_dataset
from PIL import Image as img
import matplotlib.pyplot as plt
from google.colab import files
import cv2
import numpy as np
import pytesseract
from pdf2image import convert_from_path
import pdfplumber
from google.colab import files
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

In [12]:
from google.colab import userdata
key=userdata.get('hugging_face_key')

In [13]:
login(token=key)

In [48]:
# dataset1 = load_dataset('mychen76/invoices-and-receipts_ocr_v1')
# dataset2 = load_dataset('mychen76/invoices-and-receipts_ocr_v2')

In [ ]:
# uploaded = files.upload()

In [15]:
# invoice = img.open('Airinvoice.jpg')

In [41]:
# invoice

In [17]:
# pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'
# Preprocessing Function
def preprocess_image(image):
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)
    denoised = cv2.fastNlMeansDenoising(gray, None, 30, 7, 21)
    clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8, 8))
    contrast_enhanced = clahe.apply(denoised)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    cleaned = cv2.morphologyEx(contrast_enhanced, cv2.MORPH_CLOSE, kernel)
    thresh = cv2.adaptiveThreshold(cleaned, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY, 11, 2)
    return thresh


def extract_text_from_pdf(pdf_path):
    images = convert_from_path(pdf_path, dpi=300 ,poppler_path="/usr/bin" )

    extracted_text = ""

    for i, image in enumerate(images):
        processed_image = preprocess_image(image)
        custom_config = '--psm 6 --oem 3'
        page_text = pytesseract.image_to_string(processed_image, config=custom_config)
        extracted_text += f"\n--- Page {i + 1} ---\n{page_text}"

    print("Extracted Text:\n", extracted_text)


#Dataset pdf
pdf_path = "/content/invoice_Yana Sorensen_5434.pdf"
#OCR extraction
text=extract_text_from_pdf(pdf_path)
print(text)

Processing page 1/1...
Extracted Text:
 
--- Page 1 ---
~vpewere INVOICE
# 5434
Date: May 31 2012
Bill To: Ship To: .
Ship Mode: Standard Class
Yana Sorensen Matagalpa, — ee
Matagalpa, Nicaragua Balance Due: $1,031.66
fem @vantity, Rate)
Dania Library with Doors, Metal 2 $482.48 $964.96
Bookcases, Furniture, FUR-BO-3901
Subtotal: $964.96
Shipping: $66.70
Total: $1,031.66
Notes:
Thanks for your business!
Terms:
Order ID : MX-2012-YS2188093-41060

None


In [47]:

# def extract_clean_text(pdf_path):
#     clean_text = ""
#     with pdfplumber.open(pdf_path) as pdf:
#         for page_num, page in enumerate(pdf.pages):
#             text = page.extract_text()
#             if text:
#                 clean_text += text.strip() + "\n\n"
#     print(clean_text)
#     with open("clean_text_output.txt", "w", encoding="utf-8") as file:
#         file.write(clean_text)

# # Path to PDF
# pdf_path = "/content/invoice_Yana Sorensen_5434.pdf"

# # Run Extraction
# extract_clean_text(pdf_path)


In [49]:
# import pdfplumber
# import pytesseract
# from pdf2image import convert_from_path
# import cv2
# import numpy as np

# # Preprocessing Function for OCR (Improved)
# def preprocess_image(image):
#     gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)
#     sharpened = cv2.filter2D(gray, -1, np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]))
#     denoised = cv2.fastNlMeansDenoising(sharpened, None, 30, 7, 21)
#     clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8, 8))
#     contrast_enhanced = clahe.apply(denoised)
#     return contrast_enhanced

# # Extract Raw Text Using pytesseract
# def extract_text_with_ocr(pdf_path):
#     extracted_text = ""
#     images = convert_from_path(pdf_path, dpi=300, poppler_path="/usr/bin")

#     for i, image in enumerate(images):
#         processed_image = preprocess_image(image)
#         custom_config = '--psm 3 --oem 3'  # `--psm 3` for dense text with tables
#         page_text = pytesseract.image_to_string(processed_image, config=custom_config)
#         extracted_text += f"--- Page {i + 1} ---\n{page_text.strip()}\n\n"

#     print("\n>>> Raw Text from OCR <<<\n")
#     print(extracted_text)
#     with open("raw_text_output.txt", "w", encoding="utf-8") as file:
#         file.write(extracted_text)

# #  Table Extraction Using pdfplumber
# def extract_tables_from_pdf(pdf_path):
#     table_data = ""

#     # Open the PDF using pdfplumber
#     with pdfplumber.open(pdf_path) as pdf:
#         for page_num, page in enumerate(pdf.pages):
#             # Extract tables with more precise handling
#             print(f"Processing Page {page_num + 1}...\n")

#             # Adjust table settings (horizontal/vertical strategy)
#             tables = page.extract_tables(
#                 table_settings={
#                     "snap_tolerance": 10,
#                     "join_tolerance": 10,  # Merges fragmented rows
#                     "edge_min_length": 50, # Filters out small text artifacts
#                     "horizontal_strategy": "lines",  # Try extracting text as lines
#                     "vertical_strategy": "lines",  # Strategy to align vertically
#                 }
#             )

#             # Display raw text for debugging (can help identify layout issues)
#             print("Extracted Raw Text for Debugging:\n")
#             print(page.extract_text())

#             # Check if any tables were found
#             if tables:
#                 table_data += f"\n--- Tables from Page {page_num + 1} ---\n"
#                 for table in tables:
#                     for row in table:
#                         # Clean the row and join with separators
#                         cleaned_row = [str(cell).strip() if cell else '' for cell in row]
#                         table_data += " | ".join(cleaned_row) + "\n"
#             else:
#                 print("No table found on this page.")

#     # Display the extracted table data
#     print("\n Extracted Table Data \n")
#     print(table_data)

#     # Optionally, save the extracted table data to a text file
#     with open("table_data_output.txt", "w", encoding="utf-8") as file:
#         file.write(table_data)


# # Combined Function to Follow OCR → Table Flow
# def extract_data(pdf_path):
#     print("(OCR)")
#     extract_text_with_ocr(pdf_path)

#     print("\nTables (pdfplumber)")
#     extract_tables_from_pdf(pdf_path)

# # Path to PDF
# pdf_path = "/content/invoice_Yana Sorensen_5434.pdf"

# # Run Extraction
# extract_data(pdf_path)


In [54]:
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
import cv2
import numpy as np

#upload file
def upload_pdf():
    uploaded = files.upload()
    pdf_path = next(iter(uploaded))
    return pdf_path

#preprocess the image
def preprocess_image(image):
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)
    sharpened = cv2.filter2D(gray, -1, np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]))
    denoised = cv2.fastNlMeansDenoising(sharpened, None, 30, 7, 21)
    clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8, 8))
    contrast_enhanced = clahe.apply(denoised)
    return contrast_enhanced


#OCR Extraction
def extract_text_with_ocr(pdf_path):
    extracted_text = ""
    images = convert_from_path(pdf_path, dpi=300, poppler_path="/usr/bin")

    for i, image in enumerate(images):
        processed_image = preprocess_image(image)
        custom_config = '--psm 3 --oem 3'
        page_text = pytesseract.image_to_string(processed_image, config=custom_config)
        extracted_text += f"--- Page {i + 1} ---\n{page_text.strip()}\n\n"

    return extracted_text

# Table Extraction Using pdfplumber
def extract_tables_from_pdf(pdf_path):
    table_data = ""

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):

            # Adjust table settings
            tables = page.extract_tables(
                table_settings={
                    "snap_tolerance": 10,
                    "join_tolerance": 10,  # Merges fragmented rows
                    "edge_min_length": 50, # Filters out small text artifacts
                    "horizontal_strategy": "lines",  # Try extracting text as lines
                    "vertical_strategy": "lines",  # Strategy to align vertically
                }
            )


            if tables:
                table_data += f"\n--- Tables from Page {page_num + 1} ---\n"
                for table in tables:
                    for row in table:

                        cleaned_row = [str(cell).strip() if cell else '' for cell in row]
                        table_data += " | ".join(cleaned_row) + "\n"
            else:
                print("No table found on this page.")

    return table_data


def extract_data(pdf_path):
    print("(OCR)")
    ocr_text = extract_text_with_ocr(pdf_path)

    print("\nTables")
    table_data = extract_tables_from_pdf(pdf_path)

    return ocr_text, table_data


#Testing
# pdf_path = "/content/invoice_Yana Sorensen_5434.pdf"
pdf_path=upload_pdf()
ocr_text, table_data = extract_data(pdf_path)

print("\nOCR Text\n")
print(ocr_text)

print("\n Table Data Extracted from PDFPlumber\n")
print(table_data)


Saving invoice_Xylona Preis_3875.pdf to invoice_Xylona Preis_3875.pdf
(OCR)

Tables
No table found on this page.

OCR Text

--- Page 1 ---
Superstore INVOICE

# 3875
Date: May 03 2012
Bill To: Ship To:
; . Ship Mode: Standard Class
Xylona Preis Mejicanos, San
Salvador, El Salvador Balance Due: $22,940.02
Item | Quantity Bcc) Amount
Hon Wood Table, with Bottom Storage 10 $2,833.76 $28,337.60
Tables, Furniture, FUR-TA-4713
Subtotal: $28,337.60
Discount (20%): $5,667.52
Shipping: $269.94
Total: $22,940.02

Notes:
Thanks for your business!

Terms:
Order ID : MX-2012-XP21 86539-41032



 Table Data Extracted from PDFPlumber


